<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/evidence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [4]:
import pickle

with open(f"{base_path}conversations_with_sites.pkl", 'rb') as file:
    conversations = pickle.load(file)

print("Len loaded data:", len(conversations))


Len loaded data: 18184


In [5]:
def remove_characters_from_end(input_string):
    characters_to_remove = [')', '[', ')]', ').', '/', '/),']

    # Use rstrip to remove the specified characters from the end of the string
    for char in characters_to_remove:
        input_string = input_string.rstrip(char)

    return input_string

In [6]:
from urllib.parse import urlparse
import re

def find_urls_and_domains(text):
    # Define a regex pattern to match URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

    # Find all matches in the text
    urls = re.findall(url_pattern, text)

    # Extract domains from URLs using urlparse with error handling
    domains = []
    for url in urls:
        try:
            domain = urlparse(url).hostname
            domains.append((url, domain))
        except ValueError as e:
            result = url.split('](')
            for part in result:
                cleaned = remove_characters_from_end(part)
                domain = urlparse(cleaned).hostname
                domains.append((cleaned, domain))

    return domains

In [75]:
import re

def find_numbers(s):
    # Define a regex pattern to match URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

    # Find and replace all URLs in the text
    s = re.sub(url_pattern, '', s)

    # Define a regex pattern to match numbers
    #number_pattern = re.compile(r'\b(?:\d+(?:\.\d+)?|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)\b')

    # Find all matches in the text
    numbers = re.findall(r'\b(?:\d+(?:\.\d+)?|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)\b', s, flags=re.IGNORECASE)

    # Print all matches
    # for number in numbers:
    #   print(number)
    return numbers

# Test the function
find_numbers("HUNDERED are one hundred and twenty questions, but only 6 are relevant. twenty-two, Eight")


['hundred', 'twenty', '6', 'twenty', 'two', 'Eight']

In [8]:
import random

def get_random_pairs(conversations, sample_size):
  random_pairs = []

  random_sample = random.sample(conversations.items(), sample_size)
  for sample_conversation in random_sample:
      random_pair = random.choice(sample_conversation[1]['pairs'])
      for pair in sample_conversation[1]['pairs']:
        if pair['comment_id'] == random_pair['comment_id'] and pair['reply_id'] != random_pair['reply_id']:
          other_response = pair
          random_pairs.append(random_pair)
          random_pairs.append(pair)
          break
  return random_pairs

In [60]:
random_pairs =get_random_pairs(conversations, 5)

<ipython-input-8-739721b51061>:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_sample = random.sample(conversations.items(), sample_size)


In [61]:
for pair in random_pairs:
  print(pair['reply'])
  print(find_numbers(pair['reply']))

Sometimes youre just gonna have to try stuff on. It sucks and sizes are all over the place from one brand to another, ive worn stuff from medium to XXL so it really just depends unfortunately
None
Once you break the ice, you know what works. But as mentioned, you’re gonna have to try them on. 

I booked a session at my local Macy’s with a style consultant… I was scared to death but she really helped me and still does. 

Once you find a style, then a brand, things get a little more consistent. I’m 6’ 190#’s 32x32 pants. I have found that 12 dress size generally works for me. 

Woman’s pant’s all over the place. I wear 8, 10, and rarely a 12. 

If you plan to actually transition, you’re going to have to jump on in the water… it really is nice once your in…

Hope this helps
6
190
12
8
10
12
None
If you’re going to post a hot take in a meme subreddit could you at least do it the form of a meme.
None
Shut up get out. The design is hideous and the way the character is written ruined the baub

In [76]:
for key in conversations:
  for comment in conversations[key]['comments']:
    urls_and_domains = find_urls_and_domains(comment['body'])
    numbers = find_numbers(comment['body'])
    result = 0.0
    if len(urls_and_domains) != 0:
      result += 0.5
    if len(numbers) != 0:
      result += 0.5
    comment['evidence'] = result

In [63]:
domains = set()
for key in conversations:
  for comment in conversations[key]['comments']:
    urls_and_domains = find_urls_and_domains(comment['body'])
    comment['site_ref_count'] = str(len(urls_and_domains))
    #print(comment['site_ref_count'])
    #for url, domain in urls_and_domains:
    #       if domain and domain not in domains:
    #           domains.add(domain)

In [69]:
import pickle

def write_conversations_with_site():
  with open(f"{base_path}conversations_with_numbers_sites.pkl", 'wb') as file:
    pickle.dump(conversations, file)
  print("Variables written to the file.")

In [77]:
write_conversations_with_site()

Variables written to the file.
